# Import Packages and Functions

In [1]:
#Import required libraries

import warnings
warnings.filterwarnings('ignore')
from io import StringIO
import pandas as pd
import re
import numpy as np
from nltk import word_tokenize
import string
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
import sys  
!{sys.executable} -m pip install contractions
import contractions
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ajay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ajay\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ajay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Getting Data from the .csv file


# Use the the path of the .csv file, that I have attached along with the code
tweets_train_dataset=pd.read_csv("C:/Users/Ajay/Desktop/Hatespeech/FinalDataset_Train1.csv")
# tweets_train_dataset
tweets_test_dataset=pd.read_csv("C:/Users/Ajay/Desktop/Hatespeech/FinalDataset_Test1.csv")
tweets_train_dataset

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,label,tweet
0,0,43636,43636,1,If you tell a dyke she looks just like a nigga...
1,1,6047,6047,0,i am joy. #i_am #positive #affirmation
2,2,8555,8555,0,why the more #actor will #book the #job from...
3,3,7507,7507,0,@user @user @user i know how u feel i didn't ...
4,4,29933,29933,0,trying not to shut down but maybe #pokemon wil...
...,...,...,...,...,...
46740,46740,24534,24534,0,you are beautiful girl &amp; choice semoga ten...
46741,46741,7798,7798,0,@user evergreen college police can't park prop...
46742,46742,31654,31654,0,@user we are so and we just can't hide it! ...
46743,46743,15989,15989,0,chillax ð´ð´ #pasumpahanisland #island #be...


# Preprocessing the data

In [3]:
# Pre-processing the data

# Stemming and Lemmatization is usually done in pre-processing
# to get better results. But in our our case, these processes affects the result negatively.
# So these are not used (Code commented out)

Bag_of_Words=[]
for Key,Value in tweets_train_dataset.iterrows():
    words = Value['tweet']
    
# Removing Contractions: Contractions are words or combinations of words that are shortened by 
# dropping letters and replacing them by an apostrophe. These shortened words reverted back to their actual forms.
# e.g. don't -> do not , I'm -> I am.
    expanded_words = []    
    for word in words.split():
      # using contractions.fix to expand the shotened words
      expanded_words.append(contractions.fix(word))   
    words = ' '.join(expanded_words)
#     print(words)

# Removing special characters from the words
# e.g. vs. -> vs , sub-layers -> sub layers 
    words = re.sub(r'[^A-Za-z0-9 ]+',' ',words)

# split into words
    words=word_tokenize(words)
    
# convert to lower case
    words = [w.lower() for w in words]
    
# Remove stop words
    stop_words = nltk.corpus.stopwords.words('english')
    words = [i for i in words if not i in stop_words]
    
# Stemming: Stemming is the process of producing morphological variants of a root/base word.
# e.g. words: liking, likes, liked , likely ->  root word: like
#     ps = PorterStemmer()
#     words = [ps.stem(word) for word in words]

# Lemmatization: Lemmatization is the process of grouping together the different inflected forms
# of a word so they can be analysed as a single item. Lemmatization is 
# similar to stemming but it brings context to the words. So it links words with similar meaning to one word.
# e.g. rocks -> rock, corpora -> corpus.
#     lemmatizer = WordNetLemmatizer()
#     words = [lemmatizer.lemmatize(word) for word in words]
    
#     print(words)
    for k in words:
        if k not in Bag_of_Words:
            Bag_of_Words.append(k)
Bag_of_Words

['tell',
 'dyke',
 'looks',
 'like',
 'nigga',
 'compliment',
 'always',
 'wondered',
 'joy',
 'positive',
 'affirmation',
 'actor',
 'book',
 'job',
 'user',
 'know',
 'feel',
 'seen',
 'jocoxmp',
 'bbc',
 'parliament',
 'times',
 'still',
 'trying',
 'shut',
 'maybe',
 'pokemon',
 'help',
 '90skid',
 'bihday',
 'tomorrow',
 'june',
 '19',
 'crhedrys',
 'pussy',
 'licking',
 'meow',
 'stopwhitepeople2014',
 '128049',
 'https',
 'co',
 'keegdcjs5k',
 '8221',
 '128533',
 'chanelisabeth',
 'nigger',
 'music',
 'nearly',
 'folks',
 'nervous',
 'curious',
 'finalised',
 'stuff',
 'pointers',
 'grainne',
 'bringiton',
 'rt',
 'faz',
 'convinced',
 'hitler',
 'faggit',
 'bitches',
 'kids',
 'leather',
 'uniforms',
 'wanted',
 'alone',
 'closest',
 'men',
 'lock',
 '8230',
 'lit',
 'fire',
 'cheer',
 'cold',
 'damn',
 'girl',
 'porch',
 'monkey',
 'smh',
 'sad',
 'real',
 'got',
 'find',
 'negative',
 'lbj',
 'made',
 'history',
 '3',
 '1',
 'bk',
 '2',
 '41s',
 'triple',
 'double',
 'zone',


In [4]:
print("Total number of Words:",len(Bag_of_Words))

Total number of Words: 56733


# Vectorization using Count Vectorizer

In [5]:
# Vectorization using Count Vectorizer
Vector_tweets= CountVectorizer()
Vector_tweets = Vector_tweets.fit(Bag_of_Words)

In [6]:
#Train data
X = tweets_train_dataset['tweet']
Y = tweets_train_dataset['label']
X = np.array(X)
Y = np.array(Y)

# Training the Machine Learning Models

In [7]:
# Method to implement Machine learning models
# If train_accuracy is 1 then method is called to compute training accuracy, else testing accuracy
def Machine_Learning_Models(X_train,y_train,X_test,y_test,train_accuracy):
    Final_pred= Vector_tweets.transform(X_train)
    Final_pred
    Accuracies_of_All_methods = []
    # K-Nearest Neighbor Classifier
    Knn = KNeighborsClassifier(n_neighbors=1)
    Knn.fit(Final_pred,y_train)
    X_test = Vector_tweets.transform(X_test)
    prediction = Knn.predict(X_test)
    Accuracies_of_All_methods.append(accuracy_score(y_test,prediction))
    if train_accuracy == 1:
        print("K-Nearest Neighbor Classifier done")
    else:
        print("K-Nearest Neighbor Classifier :-")
        print("Confusion Matrix")
        print(confusion_matrix(y_test,prediction))
        print("Classification Report")
        print(classification_report(y_test,prediction))
    
    # Support Vector Machine Classifier
    Support_Vector_Machine = SVC()
    Support_Vector_Machine.fit(Final_pred,y_train)
    prediction = Support_Vector_Machine.predict(X_test)
    Accuracies_of_All_methods.append(accuracy_score(y_test,prediction))
    if train_accuracy == 1:
        print("Support Vector Machine Classifier done")
    else:
        print("Support Vector Machine Classifier :-")
        print("Confusion Matrix")
        print(confusion_matrix(y_test,prediction))
        print("Classification Report")
        print(classification_report(y_test,prediction))
    
    # Multinomial Naive Bayes classifier
    Multinomial_Naive_Bayes = MultinomialNB()
    Multinomial_Naive_Bayes.fit(Final_pred,y_train)
    prediction = Multinomial_Naive_Bayes.predict(X_test)
    Accuracies_of_All_methods.append(accuracy_score(y_test,prediction))
    if train_accuracy == 1:
        print("Multinomial Naive Bayes classifier done")
    else:
        print("Multinomial Naive Bayes classifier :-")
        print("Confusion Matrix")
        print(confusion_matrix(y_test,prediction))
        print("Classification Report")
        print(classification_report(y_test,prediction))
    
    # Logistic Regression Classifier
    Logistic_Regression = LogisticRegression()
    Logistic_Regression.fit(Final_pred,y_train)
    prediction = Logistic_Regression.predict(X_test)
    Accuracies_of_All_methods.append(accuracy_score(y_test,prediction))
    if train_accuracy == 1:
        print("Logistic Regression Classifier done")
        return Accuracies_of_All_methods
    else:
        print("Logistic Regression Classifier :-")
        print("Confusion Matrix")
        print(confusion_matrix(y_test,prediction))
        print("Classification Report")
        print(classification_report(y_test,prediction))
    print("Test Accuracy of Models:-")
    print("KNN : ",Accuracies_of_All_methods[0])
    print("SVM : ",Accuracies_of_All_methods[1])
    print("Naive Bayes : ",Accuracies_of_All_methods[2])
    print("Logistic Regression : ",Accuracies_of_All_methods[3])

# Using 5 Fold Cross Validation

In [8]:
# 5 Fold cross validation is implemented to find testing accuracy

# Fold 1
print("Fold1")
X_train1 = X[9349:]
Y_train1 = Y[9349:]
X_val1 = X[:9349]
Y_val1 = Y[:9349]

fold1 = Machine_Learning_Models(X_train1, Y_train1, X_val1, Y_val1,1)
print(fold1)

Fold1
K-Nearest Neighbor Classifier done
Support Vector Machine Classifier done
Multinomial Naive Bayes classifier done
Logistic Regression Classifier done
[0.8154882875173816, 0.9423467750561557, 0.918921809819232, 0.9452347844689272]


In [9]:
# Fold2
print("Fold 2")
fold2= []
X_train2 = np.concatenate((X[:9349], X[18698:]), axis=0)
Y_train2 = np.concatenate((Y[:9349], Y[18698:]), axis=0)
X_val2 = X[9349:18698]
Y_val2 = Y[9349:18698]

fold2 = Machine_Learning_Models(X_train2, Y_train2, X_val2, Y_val2,1)
print(fold2)

Fold 2
K-Nearest Neighbor Classifier done
Support Vector Machine Classifier done
Multinomial Naive Bayes classifier done
Logistic Regression Classifier done
[0.8879024494598353, 0.9399935822013049, 0.9172103968338859, 0.9422398117445716]


In [10]:
# Fold 3
print("Fold 3")
fold3= []
X_train3 = np.concatenate((X[:18698], X[28047:]), axis=0)
Y_train3 = np.concatenate((Y[:18698], Y[28047:]), axis=0)
X_val3 = X[18698:28047]
Y_val3 = Y[18698:28047]

fold3 = Machine_Learning_Models(X_train3, Y_train3, X_val3, Y_val3,1)
print(fold3)

Fold 3
K-Nearest Neighbor Classifier done
Support Vector Machine Classifier done
Multinomial Naive Bayes classifier done
Logistic Regression Classifier done
[0.8870467429671622, 0.9368916461653652, 0.9132527543052733, 0.9413841052518986]


In [11]:
# Fold 4
print("Fold 4")
fold4= []
X_train4 = np.concatenate((X[:28047], X[37396:]), axis=0)
Y_train4 = np.concatenate((Y[:28047], Y[37396:]), axis=0)
X_val4 = X[28047:37396]
Y_val4 = Y[28047:37396]

fold4 = Machine_Learning_Models(X_train4, Y_train4, X_val4, Y_val4,1)
print(fold4)

Fold 4
K-Nearest Neighbor Classifier done
Support Vector Machine Classifier done
Multinomial Naive Bayes classifier done
Logistic Regression Classifier done
[0.8950689913359717, 0.9396726922665526, 0.9225585624130923, 0.9461974542731842]


In [12]:
# Fold 5
print("Fold 5")
X_train5 = X[:37396]
Y_train5 = Y[:37396]
X_val5 = X[37396:]
Y_val5 = Y[37396:]

fold5 = Machine_Learning_Models(X_train5, Y_train5, X_val5, Y_val5,1)
print(fold5)

Fold 5
K-Nearest Neighbor Classifier done
Support Vector Machine Classifier done
Multinomial Naive Bayes classifier done
Logistic Regression Classifier done
[0.8938923949085463, 0.9435233714835811, 0.9242699753984384, 0.9472670873890255]


In [13]:
print("Average Training accuracy of KNN:", (fold1[0]+fold2[0]+fold3[0]+fold4[0]+fold5[0])/5)
print("Average Training accuracy of SVM:", (fold1[1]+fold2[1]+fold3[1]+fold4[1]+fold5[1])/5)
print("Average Training accuracy of Naive Bayes:", (fold1[2]+fold2[2]+fold3[2]+fold4[2]+fold5[2])/5)
print("Average Training accuracy of Logistic Regression:", (fold1[3]+fold2[3]+fold3[3]+fold4[3]+fold5[3])/5)

Average Training accuracy of KNN: 0.8758797732377793
Average Training accuracy of SVM: 0.940485613434592
Average Training accuracy of Naive Bayes: 0.9192426997539844
Average Training accuracy of Logistic Regression: 0.9444646486255215


# Testing the data

In [14]:
# Test dataset taken to calculate Test accuracy
X_test = tweets_test_dataset['tweet']
y_test = tweets_test_dataset['label']

In [15]:
Machine_Learning_Models(X,Y,X_test,y_test,0)

K-Nearest Neighbor Classifier :-
Confusion Matrix
[[5460  279]
 [ 776 3485]]
Classification Report
              precision    recall  f1-score   support

           0       0.88      0.95      0.91      5739
           1       0.93      0.82      0.87      4261

    accuracy                           0.89     10000
   macro avg       0.90      0.88      0.89     10000
weighted avg       0.90      0.89      0.89     10000

Support Vector Machine Classifier :-
Confusion Matrix
[[5596  143]
 [ 430 3831]]
Classification Report
              precision    recall  f1-score   support

           0       0.93      0.98      0.95      5739
           1       0.96      0.90      0.93      4261

    accuracy                           0.94     10000
   macro avg       0.95      0.94      0.94     10000
weighted avg       0.94      0.94      0.94     10000

Multinomial Naive Bayes classifier :-
Confusion Matrix
[[5228  511]
 [ 339 3922]]
Classification Report
              precision    recall  f1-sc